In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,auc, roc_auc_score
from sklearn.pipeline import make_pipeline

In [51]:
import pandas as pd 

title = ['UserID', 'ItemID', 'CategoryID', 'Action', 'Timestamp']
data = pd.read_csv("data.csv",names=title)


In [94]:
data.head()

,UserID,ItemID,CategoryID,Action,Timestamp
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146


In [42]:
# All - Feature
buy = data.loc[data['Action'] == "buy"]
buy['label'] = 1
pv = data.loc[data['Action'] == "pv"]
pv['label'] = 0
#pv.head()
fav = data.loc[data['Action'] == "fav"]
#fav.head()
fav['label'] = 0
cart = data.loc[data['Action'] == "cart"]
cart['label'] = 0
#cart.groupby('label').count()
#cart.head()

postive_fav = fav.sample(frac=0.2,random_state=1)
postive_fav['label'] = 1
postive_cart = cart.sample(frac=0.4,random_state=2)
postive_cart['label'] = 1

fav = fav.append(postive_fav)
fav.drop_duplicates(subset=['ItemID','Timestamp'],keep='last',inplace=True)
#fav.groupby('label').count()
cart = cart.append(postive_cart)
cart.drop_duplicates(subset=['ItemID','Timestamp'],keep='last',inplace=True)
#cart.groupby('label').count()


frames = [buy, pv, fav,cart] 
all_data = pd.concat(frames, sort=False)
#all_data.groupby('label').count()
# 总样本数2000000条。
# 其中负样本1903846，正样本96154
# 负样本是正样本的 19.79 倍，

all_data = all_data.sort_values("Timestamp",ascending=True)
#all_data.head()
all_data = all_data.drop(index=1928412)
#all_data.head()


label_data = all_data.sort_values("label",ascending=True)
nagetive_label = label_data[:1903845]
postive_label = label_data[1903845:]

nagetive_label = nagetive_label.sample(frac=0.052,random_state=1)
#nagetive_label.groupby('label').count()


frames = [nagetive_label,postive_label] 
all_data = pd.concat(frames,sort=False)
all_data = all_data.sort_values("Timestamp",ascending=True)
all_data.groupby('label').count()


/jxy8/jxy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/jxy8/jxy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/jxy8/jxy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,UserID,ItemID,CategoryID,Action,Timestamp
label,,,,,
0,99000,99000,99000,99000,99000
1,96154,96154,96154,96154,96154


In [43]:
X = all_data[['UserID', 'ItemID', 'CategoryID', 'Timestamp']]
y = all_data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=False)
# X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train,
#                                                             y_train,
#                                                             test_size=0.5)

In [50]:
grd = GradientBoostingClassifier(n_estimators=100,random_state=1,learning_rate=0.02,max_leaf_nodes=64,subsample=0.8)
grd_enc = OneHotEncoder()
grd_lr = LogisticRegression()

grd.fit(X_train, y_train)
grd_enc.fit(grd.apply(X_train)[:, :, 0])
grd_lr.fit(grd_enc.transform(grd.apply(X_train)[:, :, 0]), y_train)

y_pred_grd_lr = grd_lr.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0]))[:, 1]

fpr_grd_lr, tpr_grd_lr,_  = roc_curve(y_test, y_pred_grd_lr)
auc = roc_auc_score(y_test, y_pred_grd_lr) 
print("GDBT+LR:", auc)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.02, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=64,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=1, subsample=0.8, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

/jxy8/jxy/.local/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

/jxy8/jxy/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

GDBT+LR: 0.5738699190977303


In [52]:
# BP - Feature
bp_data = data
size_mapping = {"pv":0,"buy":1,'fav':0,'cart':0}
bp_data["Action"] = bp_data["Action"].map(size_mapping)
bp_data = bp_data.drop(index=1928412)

In [53]:
bp_data.head()

,UserID,ItemID,CategoryID,Action,Timestamp
0,1,2268318,2520377,0,1511544070
1,1,2333346,2520771,0,1511561733
2,1,2576651,149192,0,1511572885
3,1,3830808,4181361,0,1511593493
4,1,4365585,2520377,0,1511596146


In [54]:
bp_data.groupby('Action').count()
# 总样本数2000000条。
# 其中负样本1959757，正样本40243
# 负样本是正样本的 48.69 倍，

bp_data = bp_data.sort_values("Timestamp",ascending=True)


bp_data = bp_data.sort_values("Action",ascending=True)
nagetive_label = bp_data[:1959756]
postive_label = bp_data[1959756:]
nagetive_label.groupby('Action').count()
postive_label.groupby('Action').count()

nagetive_label = nagetive_label.sample(frac=0.022,random_state=1)
nagetive_label.groupby('Action').count()


frames = [nagetive_label,postive_label] 
bp_data = pd.concat(frames,sort=False)
bp_data = bp_data.sort_values("Timestamp",ascending=True)
bp_data.groupby('Action').count()


,UserID,ItemID,CategoryID,Timestamp
Action,,,,
0,1959756,1959756,1959756,1959756
1,40243,40243,40243,40243


,UserID,ItemID,CategoryID,Timestamp
Action,,,,
0,1959756,1959756,1959756,1959756


,UserID,ItemID,CategoryID,Timestamp
Action,,,,
1,40243,40243,40243,40243


,UserID,ItemID,CategoryID,Timestamp
Action,,,,
0,43115,43115,43115,43115


,UserID,ItemID,CategoryID,Timestamp
Action,,,,
0,43115,43115,43115,43115
1,40243,40243,40243,40243


In [55]:
X = bp_data[['UserID', 'ItemID', 'CategoryID', 'Timestamp']]
y = bp_data["Action"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=False)
X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train,
                                                            y_train,
                                                            test_size=0.5)

In [58]:
grd = GradientBoostingClassifier(n_estimators=100,random_state=1,learning_rate=0.02,max_leaf_nodes=64,subsample=0.8)
grd_enc = OneHotEncoder()
grd_lr = LogisticRegression()
grd.fit(X_train, y_train)
grd_enc.fit(grd.apply(X_train)[:, :, 0])
grd_lr.fit(grd_enc.transform(grd.apply(X_train_lr)[:, :, 0]), y_train_lr)
y_pred_grd_lr = grd_lr.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0]))[:, 1]
fpr_grd_lr, tpr_grd_lr, _ = roc_curve(y_test, y_pred_grd_lr)
auc = roc_auc_score(y_test, y_pred_grd_lr) 
print("GDBT+LR:", auc)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.02, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=64,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=1, subsample=0.8, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

/jxy8/jxy/.local/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

/jxy8/jxy/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

GDBT+LR: 0.636004704584509
